In [3]:
import pandas as pd
import numpy as np
from operator import attrgetter

In [4]:
#Задание № 1

In [5]:
file_path = "/mnt/HC_Volume_18315164/home-jupyter/jupyter-aleksandr-ladejsch-6e962/shared/problem1-reg_data.csv"

In [6]:
file_path2 = "/mnt/HC_Volume_18315164/home-jupyter/jupyter-aleksandr-ladejsch-6e962/shared/problem1-auth_data.csv"

In [7]:
reg_data=pd.read_csv(file_path,sep=';')
auth_data=pd.read_csv(file_path2,sep=';')

In [51]:
reg_data.head()


,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [52]:
reg_data.dtypes

reg_ts    int64
uid       int64
dtype: object

In [48]:
reg_data.isnull().sum()


reg_ts    0
uid       0
dtype: int64

In [49]:
reg_data.duplicated().sum()

0

In [53]:
auth_data.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [54]:
auth_data.dtypes

auth_ts    int64
uid        int64
dtype: object

In [58]:
auth_data.isnull().sum()


auth_ts    0
uid        0
dtype: int64

In [61]:
auth_data.duplicated().sum()

0

In [40]:

def cohort_analysis(reg_data, auth_data, 
                   start_date=None, end_date=None
                   ):
  
    reg_data=pd.read_csv(file_path,sep=';')
    auth_data=pd.read_csv(file_path2,sep=';')
    
    reg_data["reg_ts"] = pd.to_datetime(reg_data["reg_ts"], origin='unix', unit="s")
    auth_data["auth_ts"] = pd.to_datetime(auth_data["auth_ts"], origin='unix', unit="s")
    
    
    df_merged = pd.merge(reg_data, auth_data, how='inner', on='uid')
    
   
    if start_date:
        df_merged = df_merged[df_merged['reg_ts'] >= start_date]
    if end_date:
        df_merged = df_merged[df_merged['reg_ts'] <= end_date]
    
   
    df_merged['cohort'] = df_merged['reg_ts'].dt.to_period('D')
    df_merged['auth_period'] = df_merged['auth_ts'].dt.to_period('D')
    
   
    df_merged['period'] = (df_merged['auth_period'] - df_merged['cohort']).apply(attrgetter('n'))
    
   
    cohort_data = df_merged.groupby(['cohort', 'period']).agg(
        n_users=('uid', 'nunique')
    ).reset_index()
    
   
    cohort_pivot = cohort_data.pivot_table(
        index='cohort',
        columns='period',
        values='n_users',
        aggfunc='sum'
    ).fillna(0)
    

    cohort_size = cohort_pivot.iloc[:, 0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis=0) * 100
    if 0 in retention_matrix.columns:
        retention_matrix = retention_matrix.drop(0, axis=1)
    
   
    return retention_matrix

In [41]:
retention=cohort_analysis(reg_data, auth_data, start_date= '2020-09-07', end_date='2020-09-22')

In [42]:
retention

period,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
cohort,,,,,,,,,,,,,,,,
2020-09-07,2.935665,4.247345,5.371643,5.059338,6.371018,7.307933,5.246721,4.122423,5.621487,5.996252,5.683948,4.559650,5.059338,4.247345,5.059338,3.372892
2020-09-08,1.745636,3.927681,3.927681,4.862843,7.605985,7.543641,6.608479,5.174564,5.174564,6.546135,5.361596,5.922693,5.174564,4.738155,3.054863,0.000000
2020-09-09,1.742377,3.235843,4.667082,5.724953,6.658370,5.724953,5.911637,4.355943,5.040448,3.733665,5.040448,4.791537,4.853765,2.426882,0.000000,0.000000
2020-09-10,2.050963,4.288378,4.785581,4.226227,6.774394,6.277191,7.085146,4.847732,4.536979,5.655687,4.723431,5.220634,3.604723,0.000000,0.000000,0.000000
2020-09-11,1.302730,4.032258,4.528536,5.645161,5.707196,6.699752,6.327543,4.404467,4.652605,5.397022,5.334988,3.287841,0.000000,0.000000,0.000000,0.000000
2020-09-12,1.920694,3.841388,4.708798,4.646840,6.071871,6.567534,5.762082,5.266419,5.142503,5.700124,2.540273,0.000000,0.000000,0.000000,0.000000,0.000000
2020-09-13,1.977750,4.079110,4.264524,5.129790,6.551298,6.489493,4.944376,4.697157,4.944376,3.584672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-09-14,2.223595,3.829524,4.508956,5.311921,6.485485,6.979617,6.300185,4.447190,2.903027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-09-15,2.218115,4.066543,4.559458,6.038201,5.606901,6.900801,5.668515,2.464572,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
#Задание № 2

In [65]:
import requests
from urllib.parse import urlencode
from scipy.stats import ttest_ind, chi2_contingency

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/SOkIsD5A8xlI7Q'  


final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']


download_response = requests.get(download_url)
with open('Проект_1_Задание_2.csv', 'wb') as f:   
    f.write(download_response.content)

In [66]:
df=pd.read_csv('Проект_1_Задание_2.csv',sep=';')

In [67]:
control_group=df.query('testgroup == "a"')

In [68]:
test_group=df.query('testgroup == "b"')

In [69]:
ARPU_test = test_group['revenue'].sum() / test_group['user_id'].nunique()
ARPU_control = control_group['revenue'].sum() / control_group['user_id'].nunique()
ARPU_dif = (ARPU_test - ARPU_control) / ((ARPU_test + ARPU_control)/2)  * 100
print(f"ARPU тестовой группы: {ARPU_test:.2f}")
print(f"ARPU контрольной группы: {ARPU_control:.2f}")
print(f"Разница в ARPU : {ARPU_dif:.2f} %")

ARPU тестовой группы: 26.75
ARPU контрольной группы: 25.41
Разница в ARPU : 5.13 %


ARPU тестовой группы больше на 5.13 % .

In [70]:
n_control = len(control_group)
n_test = len(test_group)
payers_control = (control_group['revenue'] > 0).sum()
payers_test = (test_group['revenue'] > 0).sum()



In [71]:
cr_control = payers_control / n_control * 100
cr_test = payers_test / n_test * 100

In [72]:
print(f"Conversion Rate контрольной группы: {cr_control:.2f} %")
print(f"Conversion Rate тестовой группы: {cr_test:.2f} %")

Conversion Rate контрольной группы: 0.95 %
Conversion Rate тестовой группы: 0.89 %


Конверсия в платящих пользователей  контрольной группы больше чем конверсия в платящих пользователей тестовой группы.


In [73]:
ARPPU_control=control_group['revenue'].sum()/payers_control
ARPPU_test=test_group['revenue'].sum()/payers_test

In [74]:
print(f"ARPPU тестовой группы: {ARPPU_test:.2f}")
print(f"ARPPU контрольной группы: {ARPPU_control:.2f}")

ARPPU тестовой группы: 3003.66
ARPPU контрольной группы: 2664.00


В тестовой группе платящие пользователи тратят больше чем в контрольной группе.

Обоснование использования статистических тестов:
при достаточно больших выборках можно полагаться на центральную предельную теорему, которая гарантирует, что распределение средних будет стремиться к нормальному. Это позволяет использовать t‑тест для сравнения средних значений, даже если исходные данные не распределены нормально.

In [85]:
t_stat_arpu, p_val_arpu = ttest_ind(control_group['revenue'], test_group['revenue'], equal_var=False)

In [86]:
print(f"t-статистика ARPU: {t_stat_arpu}")
print(f"p-value ARPU: {p_val_arpu}")

t-статистика ARPU: -0.6234876562792212
p-value ARPU: 0.5329647014130866


In [98]:
revenue_control_paying = control_group[control_group["revenue"] > 0]["revenue"]
revenue_test_paying = test_group[test_group["revenue"] > 0]["revenue"]
t_stat_arppu, p_val_arppu = ttest_ind(revenue_control_paying, revenue_test_paying, equal_var=False)

In [99]:
print(f"t-статистика ARPPU: {t_stat_arppu}")
print(f"p-value ARPPU: {p_val_arppu}")

t-статистика ARPPU: -1.6446302664300838
p-value ARPPU: 0.10020780398916015


In [92]:
print(f"t-статистика ARPPU: {t_stat_arppu}")
print(f"p-value ARPPU: {p_val_arppu}")

t-статистика ARPPU: 2.1079349291072385
p-value ARPPU: 0.03503723231141361


p-value ARPU > 0.05 , что выше порогового значения. Разница статистически незначима.

для ARPPU тоже нет статистически значимых различий.

         Платящие    Неплатящие
Контроль          1928         200175
Тест              1805         200862

In [143]:
contingency_table = [
    [payers_control, n_control - payers_control],
    [payers_test, n_test - payers_test]
]
chi2_stat, chi2_p_val,_,_= chi2_contingency(contingency_table)

In [144]:
contingency_table

[[1928, 200175], [1805, 200862]]

In [146]:
print(f"Статистика хи-квадрат CR: {chi2_stat}")
print(f"p-value хи-квадрат CR: {chi2_p_val}")

Статистика хи-квадрат CR: 4.374729521260405
p-value хи-квадрат CR: 0.03647561892312613


p-value хи-квадрат CR меньше 0.05  . Разница в конверсии значима.

Результат: Статистически значимого увеличения ARPU или ARPPU в тестовой группе зафиксировано не было, тогда как конверсия в контрольной группе оказалась статистически значимо выше. Это говорит о том, что тестируемый набор предложений нельзя считать однозначно успешным, и его не рекомендуется внедрять для всей аудитории.

In [ ]:
# 3 Задание 

Базовое событие:

1)DAU (Daily Active Users) и MAU (Monthly Active Users)

2)Event Participation Rate – % активных игроков, принявших участие в событии

3)First-Time Entry – сколько игроков запустили событие хотя бы раз

4)Event Revenue – выручка, полученная в рамках события

5)Conversion Rate – % игроков, сделавших покупку ради прохождения события

6)ARPU / ARPPU по событию

7)Event Retention  – удержание участников события

Усложнение механики влияет на поведение и восприятие события игроками. Это требует расширения и переоценки набора метрик:

1)Retry Rate per Level – сколько попыток в среднем требуется на уровень

2)Failure Rate – % неудачных попыток на каждом уровне

3)Rollback Trigger Rate – % попыток, приведших к откату

4)Average Rollback Depth – на сколько уровней в среднем откатывается игрок

5)Frustration Proxy Metrics – резкий рост выходов из игры после отката, паузы, уменьшение времени сессий

6)Drop-off после отката – % игроков, завершивших участие сразу после отката

7)Complaint / Support Rate – обращения в поддержку или негатив в отзывах

Если игроки уходят из события сразу после откатов или не возвращаются на следующий день — это тревожный сигнал.
